# Timing Comparisons

In [28]:
from typing import Tuple

import torch

dt = 0.4

def scalar_dynamics(px: float, py: float, vx: float, vy: float, ux: float, uy: float
                    ) -> Tuple[float, float, float, float]:
        return px + ux * dt, py + uy * dt, ux, uy

def vector_dynamics(state: torch.Tensor, action: torch.Tensor) -> torch.Tensor:
    action = action.float()

    state_new = torch.zeros(5)
    state_new[2:4] = action  # velocity
    state_new[0:2] = state[0:2] + state_new[2:4] * dt
    state_new[4] = state[-1] + dt
    
    return state_new

A = torch.tensor([[1, 0, 0, 0, 0], 
              [0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0], 
              [0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1]]).float()
B = torch.tensor([[dt, 0, 1, 0, 0],
                  [0, dt, 0, 1, 0]]).t().float()
T = torch.tensor([0, 0, 0, 0, dt]).t().float()


def matrix_dynamics(state: torch.Tensor, action: torch.Tensor) -> torch.Tensor:
    return torch.mv(A, state) + torch.mv(B, action) + T

state = torch.rand(5).float()
action = torch.rand(2).float()

%timeit scalar_dynamics(state[0], state[1], state[2], state[3], action[0], action[1])
%timeit vector_dynamics(state, action)
%timeit matrix_dynamics(state, action)

32.4 µs ± 2.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
44.7 µs ± 869 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
11.8 µs ± 882 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [22]:
import torch

def loss_norm(x: torch.Tensor):
    loss = torch.sum(torch.norm(x, dim=1))
    gradient = torch.autograd.grad(loss, x, retain_graph=True, allow_unused=False)[0]
    
def loss_sum_of_squares(x: torch.Tensor):
    loss = torch.sum(torch.sqrt(torch.sum(x.pow(2), dim=1)))
    gradient = torch.autograd.grad(loss, x, retain_graph=True, allow_unused=False)[0]

A = torch.rand(20, 2)
A.requires_grad = True

%timeit loss_norm(A)
%timeit loss_sum_of_squares(A)

65.1 µs ± 507 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
81.1 µs ± 915 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
